# 12 : Fast Fourier Transform

This week I am going to ask you to set up an efficient implementation of the Fourier transform, one of the classic algorithms that is essential for a lot of physics data processing and simulation.  This is the kind of thing that computer scientists are very proud of, a 'Divide and Conquer' algorithm which somehow makes the problem much easier by studying in sub-sections and then bringing the results from the parts together again.  Wikipedia tells me it was invented by Gauss, and then re-invented by a couple of nuclear physicists working for the American govt, one of whom didn't actually know the application of his project.

The standard discrete Fourier transform (in 1D) is defined as:

$$
F(k) = \sum_{i=0..N-1} f(x_i) \exp \left(  - \frac{2\pi j}{N} ki\right)
$$

Where I will write $i$ as a summation index, $j$ as $\sqrt{-1}$.  $k$ is the index of values in Fourier space, called the *wavenumber*.

Every point in Fourier space at a given $k$ contains information about oscillations with a period $2\pi / k$ in the direct space, thus the point at $k=0$ is just the sum of values over $f()$, the point at $k=1$ describes oscillations of the longest period compatible with the boundary conditions, etc.

The output of a Fourier transform (and, optionally, the input) is complex-valued: points have both amplitude and phase.

To carry out a DFT na\"ively requires $\mathcal{O}(N^2)$ operations: one sum over $N$ for each $k$-point.  Unacceptable!  We use the Cooley-Tukey Fast Fourier Transform algorithm to reduce this to $\mathcal{O}(N \log N)$.  The basis of the Cooley-Tukey algorithm is to realise that the complex exponential is periodic, so there is no need to re-evaluate every term of the sum over $i$ from scratch at every $k$-point. [ https://en.wikipedia.org/wiki/Cooley–Tukey_FFT_algorithm ].  This exploitation of periodicity only works, obviously, if points are evenly spaced.

I'll reproduce the pseudocode from wikipedia, here (give or take some trivial reformatting):


## Radix-2 Cooley-Tukey FFT

fft2(*input array* f, *output array* F, *integer* N, *integer* s):  
1&nbsp;&nbsp;&nbsp;&nbsp;**if** N == 1 :<br />
2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;F[0] = f[0]<br />
3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **return**  
4&nbsp;&nbsp;&nbsp;&nbsp;fft2(f,F,N/2,2s);  
5&nbsp;&nbsp;&nbsp;&nbsp;fft2(f+s,F+N/2,N/2,2s);  
6&nbsp;&nbsp;&nbsp;&nbsp;**for** k=0 to N/2-1 **do**: !!comment: *0 to N/2-1 inclusive*<br />
7&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;p $\leftarrow$ F[k]<br />
8&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;q $\leftarrow$ $e^{-k 2\pi j/N}$ F[k+N/2]<br />
9&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;F[k]$\leftarrow$ p + q <br />
10&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;F[k+N/2]$\leftarrow$ p - q <br />
**return**

*to evaluate the function, start with $N=$number of points, $s=$1:*<br />
fft2(f, F, N, 1)


To talk through this algorithm, the first thing to notice is that it is defined recursively: the function calls itself (lines $4,5$).  This is quite neat, although not especially efficient to implement in C, as function calls are a computational overhead.  Recursive algorithms can in general be rewritten as iterative algorithms, however the Cooley-Tukey is *sooo* neat in its recursive statement that we will stick to this format.

On line $1$ we give the recursion a chance to bottom out: when the function is called on the smallest meaningful domain size, $N=1$, it returns directly.  On lines $4,5$, we recursively call the function with the 'step' set to twice it's previous value, so on the first level of the recursion we are splitting $x$ into even and odd indexed subsections of the array, and putting the outputs into the first and second halves of $X$ respectively.  When you implement this you can use some C idom to pass addresses of arrays with an offest, i.e. $\&(x[1])$ should pass the address of the array $x$ but starting at element 1.  You can also just use pointer arithmetic, and pass $x+1$: this is not adding 1 to the values of everything in $x$, it is moving the pointer $x$ along by one element. 

When the recursion returns, the for loop (lines $6..10$) collects information from the sub-sections of the input data and combines them using the symmetry of the complex exponential (basically just $\sin(-x)=-\sin(x)$, and $\cos(-x)=\cos(x)$). 

The algorithm as stated above doesn't know anything about the scales in direct or wavenumber space: we get a list of input values $f[0], f[1]... f[N-1]$ and it is implicitly assumed that they correspond to equally spaced points in direct space, something like $d=L/N$  $x=[-L/2,-L/2+d,-L/2+2d...\mathbf{0}...L/2-d]$. 

Having run this FFT algorithm and looked at the ouputs, it seems like these appear in wavenumber space $F[0],F[1]...F[N-1]$ with values of $k=[0,1,2,...N/2-1,\mathbf{N/2},1-N/2,2-N/2,3-N/2...-1]$.  Positive and negative $k$ values should be symmetrical if the input is real-valued, the order that they appear in the output array is determined by the particular FFT algorithm, or else by convention.  It is quite common to have the zero in the centre so that symmetry is more obvious when plotting, however from a quick look at this algorithm it seems to me that the k-values are packed here in the way I've just noted down. 

To really understand this, probably you should run through with a pencil to find the FFT of $\sin(2\pi x)$, evaluated at four points in direct space only: (x[0]=-0.5,x[1]=-0.25,x[2]=0,x[3]=0.25).  This is a simple waveform with a number $k=1$, so you expect a peak in the FFT at the 1st (not the 0th) value.  By symmetry you should also see a peak in the 3rd k-value.  


Note that this algorithm as presented works only for arrays sized as powers of 2, it isn't hard to generalise however we are trying to keep the code simple today.


## Complex arithmetic in C

As the FFT in general generates complex numbers I am going to introduce some datatypes for working with them.

These days there are solid implementations available in header files and libraries somewhere, but to save downloading things I am just implementing this stuff myself.

There is efficient library code for doing fft, I should probably tell you, called "FFTW3" (Fastest Fourier Transform in the West, 3).  If you are developing a large scale software project you would probably just use this, but it is too big and complex to be messing around with for a small project: it is still worth knowing how to code your own.

In [1]:
#include <stdio.h>
#include <math.h>

//define CPLX_DBL_T , a datatype to hold complex double precision numbers.
typedef struct cplx_dbl_t {
    double r; //real part
    double i; //imaginary part
} CPLX_DBL_T;

In [2]:
/* calculate the complex exponential of a real number w
       via Euler's formula : exp(i w) = cos(w) + i sin(w)*/

CPLX_DBL_T cplx_dbl_exp( double w ){
    
    CPLX_DBL_T y;
    y.r = cos( w );
    y.i = sin( w );
    return( y );
}

In [3]:
/* quick test of complex exponential */
void test_exp(){
    CPLX_DBL_T y;
    double     w;
    int        k;
    
    for( k = 0; k <= 4; k++ ){
        w = k * M_PI / 2.0;
        y = cplx_dbl_exp( w );
        //nice tidy printf statement, the "%+" means "always show sign".
        printf("exp %ipi/2 (= %.2f) : %+.2f %+.2fj\n", k, w, y.r, y.i);
    }
}
test_exp();

exp 0pi/2 (= 0.00) : +1.00 +0.00j
exp 1pi/2 (= 1.57) : +0.00 +1.00j
exp 2pi/2 (= 3.14) : -1.00 +0.00j
exp 3pi/2 (= 4.71) : -0.00 -1.00j
exp 4pi/2 (= 6.28) : +1.00 -0.00j


In [4]:
//while I'm here I might as well write add, subtract and multiply for you:
CPLX_DBL_T cplx_dbl_add( CPLX_DBL_T a, CPLX_DBL_T b ){
    CPLX_DBL_T y;
    y.r = a.r + b.r;
    y.i = a.i + b.i;
    return( y );
}

In [5]:
CPLX_DBL_T cplx_dbl_sub( CPLX_DBL_T a, CPLX_DBL_T b ){
    CPLX_DBL_T y;
    y.r = a.r - b.r;
    y.i = a.i - b.i;
    return( y );
}

In [6]:
CPLX_DBL_T cplx_dbl_mul( CPLX_DBL_T a, CPLX_DBL_T b ){
    CPLX_DBL_T y;
    y.r = a.r*b.r - a.i*b.i;
    y.i = a.r*b.i + a.i*b.r;
    return( y );
}

## Assignment: FFT

Your assignment today is to implement a code which can:

1) Load a data file, with two doubles per line (real and imaginary components) into an array of type CPLX_DBL_T. Recall the functions fopen(), fscanf(), fclose().

2) Carry out a radix-2 fft based on the pseudocode above.

3) Write the output as a data file with the same format as the inputs (recall the function fprintf()).

I've supplied an example data file for you to test your code.


In [7]:
#include <stdio.h>     //input and output to file or screen
#include <stdlib.h>    //standard.... library...
#include <iostream>

CPLX_DBL_T *get_data(void){
    FILE *f; // pointer to the file
    double rePart, imPart; // buffer for reading
    CPLX_DBL_T *data; // array of data points
    int N = 64; // amount of points
    int i;
    
    // array to store data points: y values in equidistant nodes 
    data = (CPLX_DBL_T *)malloc(N*sizeof(CPLX_DBL_T));
    
    // open the file
    f = fopen("test_data_64.txt", "r");
    
    // get the real and imaginary part from each column
    i = 0; // counter
    while (fscanf(f, "%lf %lf", &rePart, &imPart) == 2){
        data[i].r = rePart;
        data[i].i = imPart;
        // printf("real part: %+.4lf | imaginary part: %+.4lf\n", data[i].r, data[i].i);
        i += 1;
    }
    // printf("%i",i);
    
    // close the file
    fclose(f);
    
    return( data );
}

In [8]:
void FFT(CPLX_DBL_T *data, CPLX_DBL_T *results, int N, int s){
    
    CPLX_DBL_T p, q;
    
    if(N == 1){
        results[0] = data[0];
        return;
    }
    FFT(data,     results,         N / 2, 2 * s);
    FFT(data + s, results + N / 2, N / 2, 2 * s);
    for(int k = 0; k < N/2; k++){
        p = results[k];
        q = cplx_dbl_mul(cplx_dbl_exp(- k * 2 * M_PI / N) , results[k + N / 2]);
        results[k] = cplx_dbl_add(p,q);
        results[k+N/2] = cplx_dbl_sub(p,q);
    }
    return;
}

In [9]:
#include <stdio.h>     //input and output to file or screen
#include <stdlib.h>    //standard.... library...
#include <iostream>

void print2file(CPLX_DBL_T *results){
    FILE *f;
    int i;
    f = fopen("res_data_64.csv", "w"); // csv files are easier to plot in excel and qtiPlot
    
    for(i = 0; i < 65; i++){
        fprintf(f, "%lf, %lf\n", results[i].r, results[i].i);
    }
    
    fclose(f);
    return( EXIT_SUCCESS);
}

In [10]:
int main(){
    CPLX_DBL_T *data, *results;
    int N = 64;
    int s = 1;
    
    data = (CPLX_DBL_T *)malloc(N*sizeof(CPLX_DBL_T));
    results = (CPLX_DBL_T *)malloc(N*sizeof(CPLX_DBL_T));
    
    data = get_data();
    
    FFT( data, results, N, s);
    
    for(int i = 0; i < 64; i++){
        printf("real part: %+.4lf | imaginary part: %+.4lf\n", results[i].r, results[i].i);
    } 
    print2file(results);
    free(results);
    free(data);
    return( EXIT_SUCCESS);
}

In [11]:
main();

real part: +0.0000 | imaginary part: +0.0000
real part: -0.0001 | imaginary part: -32.0002
real part: -0.0001 | imaginary part: -32.0002
real part: +0.0001 | imaginary part: +0.0001
real part: +0.0000 | imaginary part: -0.0003
real part: +0.0000 | imaginary part: -0.0003
real part: +0.0002 | imaginary part: -0.0001
real part: +0.0000 | imaginary part: -0.0001
real part: -0.0001 | imaginary part: +0.0000
real part: +0.0000 | imaginary part: +0.0002
real part: -0.0001 | imaginary part: -0.0002
real part: -0.0002 | imaginary part: -0.0001
real part: +0.0000 | imaginary part: +0.0000
real part: -0.0000 | imaginary part: +0.0001
real part: -0.0000 | imaginary part: -0.0001
real part: +0.0002 | imaginary part: +0.0002
real part: +0.0001 | imaginary part: +0.0001
real part: -0.0000 | imaginary part: +0.0001
real part: +0.0001 | imaginary part: -0.0001
real part: -0.0000 | imaginary part: +0.0000
real part: -0.0002 | imaginary part: +0.0002
real part: -0.0000 | imaginary part: +0.0001
real par